In [1]:
"""
コーパスの確認をするスクリプトです。
"""
import re
import pandas as pd
import csv

csv.field_size_limit(1000000000) # python によるcsvの読み込みフィールドの制限を引き上げ
clean_text = re.compile('[!"#$＄%&\'\\\\()*+,-./:：;<=>?@[\\]^_`{|}~「」｣〔〕＜＞～“”〈〉『』【】＆＊✴・（）↑＄＃＠Ⓣ。、？…！｀＋￥％⚫︎⭕〇◯△◇☆✩☆✧･　※ ◎○▼▲■◆♦●★♡┗└┌✿ё◉♪♫♬]')

# engine = pythonにしないと、データ内のEOFに引っかかって読み込めない
data = pd.read_csv("../data/train_data/rakuten_scray_train.csv",names=["id","step"],engine="c",sep=',', error_bad_lines=False)
data = data.dropna()
cleaned_data = data.apply(lambda data:data.str.replace(clean_text,'',regex=True))
del data
print("全データ数:",len(cleaned_data))
cleaned_data.head()

/tmp/ipykernel_3424/3076381981.py:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("../data/train_data/rakuten_scray_train.csv",names=["id","step"],engine="c",sep=',', error_bad_lines=False)


全データ数: 9285470


,id,step
レシピID：1710001960,赤ワイン,12カップ
レシピID：1710001960,梅酒,12カップ
レシピID：1750080919,レタス,中2枚
レシピID：1750080919,ミックス千切り野菜,20ｇ
レシピID：1750080919,林檎,くし形中18玉


In [2]:
cleaned_data = cleaned_data.dropna(subset=["step"])
cleaned_data["str_len"] = cleaned_data["step"].apply(lambda data:len(data))
output = cleaned_data[cleaned_data["str_len"] != 0]
output.head()

,id,step,str_len
0,ad7d585b06850f8437ff5fb97d3c7a823ff21bb1,鍋に豚肉をいれて酒砂糖みりん醤油しょうが薄切りにんにくで煮込みます,33
1,4afce5687dc173ad4fef943b686582a1cd06e264,にんじんとれんこんをおろし金でおろします,20
2,4afce5687dc173ad4fef943b686582a1cd06e264,挽肉と玉ねぎのみじん切りを加えよく混ぜます,21
3,4afce5687dc173ad4fef943b686582a1cd06e264,塩コショウを加えピーマンに詰めあとは焼くだけ,22
4,4afce5687dc173ad4fef943b686582a1cd06e264,少し蒸らして火を通しできあがり,15


In [4]:
"""
コーパスの作成
"""
from tqdm import tqdm as tq
import MeCab

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

# 合計語彙数
num = 0
# データの総文字数
str_len = output["str_len"].sum()
# 学習データ
corpas = ""
# 直前のid
id_before = ""

for step in tq(output.itertuples(),total=len(output)):
    if id_before != step.id:
        corpas += "\n"
    id_before = step.id
    wakati = m.parse(step.step)
    wakati = wakati.replace("\n","")
    num += len(wakati.split(' '))
    corpas += wakati

with open("../data/train_data/cookpad_step_corpas_GROUPED_BY_ID.txt",'w') as out_corpas:
    out_corpas.write(corpas)

print("平均語彙サイズ:",str_len/num)
print("平均語彙数:",num/len(output))

100%|██████████| 8784203/8784203 [03:21<00:00, 43659.08it/s]


平均語彙サイズ: 1.8275186829316827
平均語彙数: 18.82481074264791
